In [3]:
# Load packages
import numpy as np
import pandas as pd

In [4]:
# Load data
data = pd.read_csv("data/NY property data.csv")

In [5]:
# Check if it was correctly read in
data.shape

(1070994, 32)

### 1. Remove irrelevant records

In [6]:
# List of owners to remove (government related identities are not the subject of interest)
remove_list = [
    'PARKCHESTER PRESERVAT',
    'PARKS AND RECREATION',
    'DCAS',
    'HOUSING PRESERVATION',
    'CITY OF NEW YORK',
    'DEPT OF ENVIRONMENTAL', 
    'BOARD OF EDUCATION',
    'NEW YORK CITY HOUSING',
    'CNY/NYCTA',
    'NYC HOUSING PARTNERSH',
    'DEPARTMENT OF BUSINES', 
    'DEPT OF TRANSPORTATIO',
    'MTA/LIRR',
    'PARCKHESTER PRESERVAT',
    'MH RESIDENTIAL 1, LLC', 
    'LINCOLN PLAZA ASSOCIA',
    'UNITED STATES OF AMER',
    'U S GOVERNMENT OWNRD', 
    'THE CITY OF NEW YORK',
    'NYS URBAN DEVELOPMENT',
    'NYS DEPT OF ENVIRONME',
    'CULTURAL AFFAIRS',
    'DEPT OF GENERAL SERVI',
    'DEPT RE-CITY OF NY']

In [7]:
# Remove the irrelevant owners
property_data = data[~data['OWNER'].isin(remove_list)].reset_index(drop=True)
data.shape[0] - property_data.shape[0]

24168

### 2. Fill in missing fields : ZIP

In [8]:
# Fill in missing ZIP
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

21772

In [9]:
for i in range(len(missing_zips)):
    if property_data.loc[missing_zips[i]+1, 'ZIP'] == property_data.loc[missing_zips[i]-1, 'ZIP']:
        property_data.loc[missing_zips[i], 'ZIP'] = property_data.loc[missing_zips[i]-1, 'ZIP']

In [10]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

10245

In [11]:
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i], 'ZIP'] = property_data.loc[missing_zips[i]-1, 'ZIP']

In [12]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

### 2. Fill in missing fields : FULLVAL, AVLAND, AVTOT

In [13]:
# Fill in missing FULLVAL

property_data['FULLVAL'].replace('NaN', 0)
temp = property_data[property_data['FULLVAL'] != 0]
mean_fullval = temp.groupby('TAXCLASS')['FULLVAL'].mean()
print(mean_fullval)

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64


In [14]:
# Fill in missing AVLAND 

property_data['AVLAND'].replace('NaN', 0)
temp_avland = property_data[property_data['AVLAND'] != 0]
mean_avland = temp_avland.groupby('TAXCLASS')['AVLAND'].mean()
print(mean_avland)

TAXCLASS
1      14901.860425
1A      2247.472465
1B      9185.394750
1C      8225.658898
1D    709303.793103
2      88471.102578
2A     31370.740197
2B     54607.394911
2C     25635.944099
3      28135.636364
4     444996.049088
Name: AVLAND, dtype: float64


In [15]:
# Fill in missing AVTOT 

property_data['AVTOT'].replace('NaN', 0)
temp_avtot = property_data[property_data['AVTOT'] != 0]
mean_avtot = temp_avtot.groupby('TAXCLASS')['AVTOT'].mean()
print(mean_avtot)

TAXCLASS
1     2.502549e+04
1A    1.437036e+04
1B    9.189080e+03
1C    2.898447e+04
1D    1.166866e+06
2     3.602638e+05
2A    7.963525e+04
2B    1.790961e+05
2C    1.170418e+05
3     3.829505e+04
4     1.294237e+06
Name: AVTOT, dtype: float64


In [14]:
# Fill in missing FULLVAL, AVLAND, AVTOT 

for index in mean_fullval.index:
    property_data.loc[(property_data['FULLVAL'] == 0) & (property_data['TAXCLASS']==index), 'FULLVAL'] = mean_fullval[index]
    property_data.loc[(property_data['AVLAND'] == 0) & (property_data['TAXCLASS']==index), 'AVLAND'] = mean_avland[index]
    property_data.loc[(property_data['AVTOT'] == 0) & (property_data['TAXCLASS']==index), 'AVTOT'] = mean_avtot[index]

### 2. Fill in missing fields : STORIES

In [15]:
# Fill in missing STORIES 

temp = property_data[property_data['STORIES'].isnull()]
len(temp)

43968

In [16]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [17]:
mean_stories = property_data.groupby('TAXCLASS')['STORIES'].mean()
print(mean_stories)

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64


In [18]:
property_data['STORIES'] = property_data['STORIES'].fillna(value=0)
for index in mean_stories.index:
    property_data.loc[(property_data['STORIES'] == 0) & (property_data['TAXCLASS']==index), 'STORIES'] = mean_stories[index]

In [19]:
property_data.head().T

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
B,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
EASEMENT,NaN,NaN,NaN,NaN,NaN
OWNER,U S GOVT LAND & BLDGS,U S GOVT LAND & BLDGS,"TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC"
BLDGCL,P7,Z9,R5,R5,R5
TAXCLASS,4,4,4,4,4
LTFRONT,500,27,0,0,0


### 2. Fill in missing fields: Lot and building sizes

In [20]:
# Fill in missing lot and building sizes

property_data.loc[property_data['LTFRONT'] == 0, 'LTFRONT'] = np.nan
property_data.loc[property_data['LTDEPTH'] == 0, 'LTDEPTH'] = np.nan
property_data.loc[property_data['BLDFRONT'] == 0, 'BLDFRONT'] = np.nan
property_data.loc[property_data['BLDDEPTH'] == 0, 'BLDDEPTH'] = np.nan
property_data.loc[property_data['LTFRONT'] == 1, 'LTFRONT'] = np.nan
property_data.loc[property_data['LTDEPTH'] == 1, 'LTDEPTH'] = np.nan
property_data.loc[property_data['BLDFRONT'] == 1, 'BLDFRONT'] = np.nan
property_data.loc[property_data['BLDDEPTH'] == 1, 'BLDDEPTH'] = np.nan

In [21]:
mean_LTFRONT = property_data.groupby('TAXCLASS')['LTFRONT'].mean()
mean_LTDEPTH = property_data.groupby('TAXCLASS')['LTDEPTH'].mean()
mean_BLDFRONT = property_data.groupby('TAXCLASS')['BLDFRONT'].mean()
mean_BLDDEPTH = property_data.groupby('TAXCLASS')['BLDDEPTH'].mean()

In [22]:
for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull()) & (property_data['TAXCLASS']==index), 'LTFRONT'] = mean_LTFRONT[index]
    property_data.loc[(property_data['LTDEPTH'].isnull()) & (property_data['TAXCLASS']==index), 'LTDEPTH'] = mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull()) & (property_data['TAXCLASS']==index), 'BLDFRONT'] = mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull()) & (property_data['TAXCLASS']==index), 'BLDDEPTH'] = mean_BLDDEPTH[index]

In [23]:
# Convert ZIP to string 
property_data['ZIP'] = property_data['ZIP'].astype(str)
property_data['zip3'] = property_data['ZIP'].str[:3]

### 3. Create candidate variables

In [24]:
# Create 3 size variables
property_data['ltsize'] = property_data['LTFRONT'] * property_data['LTDEPTH']
property_data['bldsize'] = property_data['BLDFRONT'] * property_data['BLDDEPTH']
property_data['bldvol'] = property_data['bldsize'] * property_data['STORIES']

In [25]:
# Create 9 variables 
property_data['r1'] = property_data['FULLVAL'] / property_data['ltsize']
property_data['r2'] = property_data['FULLVAL'] / property_data['bldsize']
property_data['r3'] = property_data['FULLVAL'] / property_data['bldvol']
property_data['r4'] = property_data['AVLAND'] / property_data['ltsize']
property_data['r5'] = property_data['AVLAND'] / property_data['bldsize']
property_data['r6'] = property_data['AVLAND'] / property_data['bldvol']
property_data['r7'] = property_data['AVTOT'] / property_data['ltsize']
property_data['r8'] = property_data['AVTOT'] / property_data['bldsize']
property_data['r9'] = property_data['AVTOT'] / property_data['bldvol']

In [26]:
# Create variables for comparison
ninevars = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9']
zip5_mean = property_data.groupby('ZIP')[ninevars].mean()
zip3_mean = property_data.groupby('zip3')[ninevars].mean()
taxclass_mean = property_data.groupby('TAXCLASS')[ninevars].mean()
borough_mean = property_data.groupby('B')[ninevars].mean()
property_data = property_data.join(zip5_mean, on='ZIP', rsuffix='_zip5')
property_data = property_data.join(zip3_mean, on='zip3', rsuffix='_zip3')
property_data = property_data.join(taxclass_mean, on='TAXCLASS', rsuffix='_taxclass')
property_data = property_data.join(borough_mean, on='B', rsuffix='_boro')
rsuffix = ['_zip5', '_zip3', '_taxclass', '_boro']
for var in ninevars: 
    for r in rsuffix:
        property_data[str(var)+r] = property_data[var] / property_data[str(var)+r]

In [27]:
property_data.columns

Index(['RECORD', 'BBLE', 'B', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro'],
      dtype='object')

In [28]:
fraud_vars = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro']
len(fraud_vars)

45

In [29]:
# Summary statistics of candidate variables
summary = property_data[fraud_vars].describe().T
#summary.to_csv('SummaryStatistics.csv')

In [30]:
property_data[fraud_vars].head(10).T

,0,1,2,3,4,5,6,7,8,9
r1,40.917782,57678.882190,385.182796,556.662267,1035.027624,98.808814,226.341598,213.990106,754.854727,734.748241
r2,3994.412456,36173.697854,736.308714,1064.105879,1978.540749,188.881206,432.670651,409.059754,1442.967127,1404.531920
r3,723.985188,6556.463990,14.726174,21.282118,39.570815,3.777624,8.653413,8.181195,28.859343,28.090638
r4,8.079350,4258.951518,62.110738,89.761892,94.470682,15.932981,36.497710,34.505770,121.720420,118.478343
r5,788.709805,2671.030012,118.729804,171.587267,180.588507,30.457208,69.768387,65.960629,232.678632,226.481133
r6,142.953243,484.122805,2.374596,3.431745,3.611770,0.609144,1.395368,1.319213,4.653573,4.529623
r7,18.413002,25955.496986,173.332268,250.498069,465.762431,44.464006,101.853749,96.295553,339.684642,330.636743
r8,1797.485605,16278.164034,331.338940,478.847739,890.343337,84.996617,194.701849,184.076898,649.335235,632.039429
r9,325.793334,2950.408795,6.626779,9.576955,17.806867,1.699932,3.894037,3.681538,12.986705,12.640789
r1_zip5,0.096913,136.610717,0.912294,1.318438,2.451432,0.234026,0.536083,0.506829,1.787851,1.740229


In [33]:
# save the data into a csv file 
df_vars = property_data[fraud_vars]
df_vars.to_csv('data_with_variables.csv')